In [78]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt

In [79]:
df = pd.read_parquet('../data/end_df.parquet')

In [80]:
df

,place,product,Our_company,cost,Арториас&Co,Длань господня,ЛилIT,Светлые Души,hot,rain,...,year,month,day,Арториас&Co_k,Длань господня_k,ЛилIT_k,Светлые Души_k,1/amount,max_price,min_price
0,Анор Лондо,Целебные травы,3.90,2.07,3.78,4.21,4.18,3.75,1.0,0.0,...,2216,1,2,1.031746,0.926366,0.933014,1.040000,0.025069,4.21,3.75
1,Анор Лондо,Целебные травы,3.90,2.07,3.78,4.21,4.18,3.75,1.0,0.0,...,2216,1,3,1.031746,0.926366,0.933014,1.040000,0.023761,4.21,3.75
2,Анор Лондо,Целебные травы,3.90,2.07,3.78,4.21,4.18,3.75,0.0,0.0,...,2216,1,4,1.031746,0.926366,0.933014,1.040000,0.026369,4.21,3.75
3,Анор Лондо,Целебные травы,3.90,2.07,3.78,4.21,4.18,3.75,0.0,0.0,...,2216,1,5,1.031746,0.926366,0.933014,1.040000,0.025747,4.21,3.75
4,Анор Лондо,Целебные травы,3.90,2.07,3.78,4.21,4.18,3.75,0.0,0.0,...,2216,1,6,1.031746,0.926366,0.933014,1.040000,0.021744,4.21,3.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,Фалькония,Эстус,9.28,3.26,15.67,14.58,14.02,15.36,0.0,1.0,...,2218,9,23,0.592214,0.636488,0.661912,0.604167,0.018365,15.67,14.02
14996,Фалькония,Эстус,9.28,7.29,15.67,14.58,14.02,15.36,0.0,0.0,...,2218,9,24,0.592214,0.636488,0.661912,0.604167,0.020541,15.67,14.02
14997,Фалькония,Эстус,9.28,7.29,15.67,14.58,14.02,15.36,0.0,0.0,...,2218,9,25,0.592214,0.636488,0.661912,0.604167,0.017403,15.67,14.02
14998,Фалькония,Эстус,9.28,7.29,15.67,14.58,14.02,15.36,0.0,0.0,...,2218,9,26,0.592214,0.636488,0.661912,0.604167,0.018094,15.67,14.02


In [81]:
df_cost = df[['place', 'product', 'cost', 'year', 'month', 'day']]
X_t, y_t = df_cost.loc[df_cost['year'] <= 2217, ['place', 'product', 'year', 'month', 'day']], df_cost.loc[df_cost['year'] <= 2217, 'cost']
X_v, y_v = df_cost.loc[df_cost['year'] > 2217, ['place', 'product', 'year', 'month', 'day']], df_cost.loc[df_cost['year'] > 2217, 'cost']

In [82]:
place_un = df['place'].unique()
product_un = df['product'].unique()
vr = []
for plac in place_un:
    for prod in product_un:
        vr.append((plac, prod))

In [83]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [104]:
class LSTML(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(LSTML, self).__init__()
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, num_layers, dropout=0.25, batch_first=True)
        self.linear = torch.nn.Linear(hidden_dim, 1)
        self.loss = torch.nn.MSELoss()
        self.optim = torch.optim.Adam(self.parameters(), lr=3e-4)

    def forward(self, x):
        x = self.lstm(x)
        x = self.linear(x)
        return x

In [105]:
model = [(  plac,\
            prod,\
            LSTML(3, 100, 5).to(device),\
            torch.tensor(X_t[(X_t['place'] == plac) & (X_t['product'] == prod)].drop(['place', 'product'], axis=1).values),\
            torch.tensor(y_t[(X_t['place'] == plac) & (X_t['product'] == prod)].values),\
            torch.tensor(X_v[(X_v['place'] == plac) & (X_v['product'] == prod)].drop(['place', 'product'], axis=1).values),\
            torch.tensor(y_v[(X_v['place'] == plac) & (X_v['product'] == prod)].values)) for plac, prod in vr]

In [108]:
for plac, prod, mod, x_train, y_train, x_val, y_val in model:
    lst_loss = []
    num_epochs = 500
    batch_size = x_train.size(0)
    x_train = x_train.view(x_train.size(0), 1, 3).float()
    print(x_train)
    for epoch in range(num_epochs):
        outputs = mod(x_train.to(device))
        loss = model.loss(outputs, y_train.to(device))
        model.optim.zero_grad()
        loss.backward()
        model.optim.step()
        with torch.no_grad():
            lst_loss.append(model.loss(x_val.to(device), y_val.to(device)).item())
    fig = plt.figure(figsize=(18,7))
    plt.plot(lst_loss)
    plt.show()

tensor([[[2.2160e+03, 1.0000e+00, 2.0000e+00]],

        [[2.2160e+03, 1.0000e+00, 3.0000e+00]],

        [[2.2160e+03, 1.0000e+00, 4.0000e+00]],

        ...,

        [[2.2170e+03, 1.2000e+01, 2.9000e+01]],

        [[2.2170e+03, 1.2000e+01, 3.0000e+01]],

        [[2.2170e+03, 1.2000e+01, 3.1000e+01]]])


TypeError: linear(): argument 'input' (position 1) must be Tensor, not tuple